In [ ]:
%pip install lonboard overturemaps

In [2]:

import overturemaps as om
from overturemaps import core
import overturemaps
import pandas
import geopandas as gpd
from shapely import wkb
from lonboard import Map, PolygonLayer, ScatterplotLayer
import ipywidgets as widgets
import numpy as np
from IPython.display import display

'''
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from huggingface_hub import HfApi, hf_hub_download
'''

'\nfrom transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification\nfrom datasets import load_dataset\nfrom huggingface_hub import HfApi, hf_hub_download\n'

In [3]:
bbox = 139.6917, 35.6895, 139.7017, 35.6995
bbox = -109.048, 37.000, -102.040, 41.000 #colorado

In [4]:
def create_map(dataset):
    layer = ScatterplotLayer.from_geopandas(
        dataset,
        get_fill_color=[0, 128, 128],
        radius_min_pixels = 1.5,
    )

    view_state = {
        "longitude": (bbox[0]+bbox[2])/2,
        "latitude": (bbox[1]+bbox[3])/2,
        "zoom": 8,
        "pitch": 0,
    }
    return Map(layer,view_state=view_state)
    

In [5]:
gdf = core.geodataframe("place", bbox=bbox)
print(np.shape(gdf))

(257710, 15)


In [ ]:
create_map(gdf)

In [6]:
# Randomly sample 10,000 points from gdf
sampled_gdf = gdf.sample(n=10000, random_state=42)
print(sampled_gdf.shape)

(10000, 15)


In [9]:
create_map(sampled_gdf)

c:\Users\iamcy\anaconda3\envs\cyrusc\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(custom_attribution='', layers=(ScatterplotLayer(get_fill_color=[0, 128, 128], radius_min_pixels=1.5, table…

In [7]:
def dist_from_address(point, address):
    """
    Calculate the distance from a point to an address.
    """
    # Convert the address to a point
    address_point = gdf[gdf['address'] == address].geometry.values[0]
    # Calculate the distance
    distance = point.distance(address_point)
    return distance

In [ ]:
def OVERTURE_address_to_coord(address):
    """
    Convert an address to coordinates.
    """
    # Get the coordinates of the address
    coords = gdf[gdf['address'] == address].geometry.values[0]
    return coords.x, coords.y

In [22]:
def CROSSREFERENCED_address_to_coord(address):
    """
    #TODO find a way to cross reference address with other dataset. 
    """

In [72]:
def check_on_road(address):
   print(f'address: {address}')
   bbox = address.x-0.0001, address.y+-.0001, address.x+0.0001, address.y+0.0001
   print(f"BBOX: {bbox}")
   roads = overturemaps.record_batch_reader("connector", bbox).read_all()
   print("Len roads: ", len(roads))
   if len(roads) > 0:
      return True
   else:
      return False
   #gdf = core.geodataframe("transportation", bbox=(0,0,0,0))

In [73]:
for i in range(len(sampled_gdf)):
    point_coords = sampled_gdf.geometry.values[i]
    #print(f'Point {i}\'s coordinates: {point_coords}')
    
    #print(f'Point {i}\'s address: {addess}')
    #print(f'Point {i}\'s coordinates: {sampled_gdf.geometry.values[i]}')
    print(f"Is address {i} on a road? {check_on_road(point_coords)}")
    # Calculate the point from the point to the address
    #distance = dist_from_address(point, "Tokyo Tower")
    #print(f"Distance from {point} to Tokyo Tower: {distance} meters")

address: POINT (-105.1094203 39.7472391)
BBOX: (-105.1095203, 39.7471391, -105.1093203, 39.747339100000005)
Len roads:  0
Is address 0 on a road? False
address: POINT (-105.0135101 39.7485743)
BBOX: (-105.01361010000001, 39.7484743, -105.0134101, 39.748674300000005)


KeyboardInterrupt: 